In [1]:
# imports
import time
import pandas as pd
import glob
import os.path
import json
import os
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define Helper Functions

In [2]:
#test if download completed
def latest_download_file(path):
    #os.chdir(path)
    #os.getcwd()
    #files = sorted(os.listdir(path), key=os.path.getmtime)
    files = sorted(os.listdir(folder_path), key= lambda file: os.path.getmtime(folder_path+'/'+file))
    newest = files[-1]
    return newest

def click_and_wait_till_download_starts(BS_Objct_to_click,path):
    pre_download_newest_file = latest_download_file(path)
    
    BS_Objct_to_click.click()
    print("waiting till download starts")
    newest_file = latest_download_file(path)
    while pre_download_newest_file == newest_file:
        time.sleep(1)
        newest_file = latest_download_file(path)
        if pre_download_newest_file != newest_file:
            print("file download detected")
        else:
            continue    

def wait_till_download_completes(path):
    fileends = "crdownload"
    start_time = time.time()
    while "crdownload" == fileends:
        time.sleep(1)
        newest_file = latest_download_file(path)
        if "crdownload" in newest_file:
            elapsed = time.time() - start_time
            print("waiting for download to complete. elapsed_time: "+str(elapsed), end="\r", flush=True)
            fileends = "crdownload"
        else:
            fileends = "none"

In [3]:
def filter_country(driver,country_name,cog_wheel,dimensions_button,country_edit_button,deselect_all_button,filter_dialogue_window,apply_button):
    cog = driver.find_element_by_xpath(cog_wheel)
    cog.click()

    dim = driver.find_element_by_xpath(dimensions_button)
    actionChains = ActionChains(driver)
    actionChains.move_to_element(dim).perform()
    actionChains.click(dim).perform()

    actionChains = ActionChains(driver)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,country_edit_button)))
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, country_edit_button)))
    country_edit = driver.find_element_by_xpath(country_edit_button)
    actionChains.move_to_element(country_edit).perform()
    actionChains.click(country_edit).perform()
    
    
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,filter_dialogue_window)))

    filter_dialogue = driver.find_element_by_xpath(filter_dialogue_window)
    actionChains = ActionChains(driver)
    actionChains.move_to_element(filter_dialogue).perform()
    actionChains.context_click(filter_dialogue).perform()

    deselect_all = driver.find_element_by_xpath(deselect_all_button)
    deselect_all.click()

    
    filter_dialogue.find_element_by_xpath(".//span[text()='{}']".format(country_name)).click()
    filter_dialogue.find_element_by_xpath(select_button).click()
    
    actionChains = ActionChains(driver)
    apply = driver.find_element_by_xpath(apply_button)
    actionChains.move_to_element(apply).perform()
    actionChains.click(apply).perform()

    cog.click()

    
    


# Define Main Function

In [4]:
def Upload_Data_as_JSON(driver,URL, country_name, loc_s,loc_c,folder_path,New_Name,SDG_bool):


    time.sleep(2)
    #Reads the URL 
    driver.get(URL)

    #Waits till the page loads
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH,loc_s)))
    WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[role="row"]')))

    if SDG_bool:
        filter_country(driver,country_name,cog_wheel,dimensions_button,country_edit_button,deselect_all_button,filter_dialogue_window,apply_button)
        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[role="row"]')))
    else:
        pass

    #finds the download button
    s = driver.find_element_by_xpath(loc_s)

    s.click()

    #Choose to export the data as excel
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,loc_c)))
    
    c=driver.find_element_by_xpath(loc_c)

    time.sleep(10)
    # to download pass c to click_and_wait_till_download_starts function
    
    click_and_wait_till_download_starts(BS_Objct_to_click=c,path=folder_path)

    wait_till_download_completes(folder_path)


    file_type = '/*xlsx'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)


    #Saving the data as dic instead of excel

    df = pd.read_excel(max_file)
    df = df.rename(lambda x: 'Country' if any(k in x for k in ['Country Name', 'Reporter', 'Country ','Country']) else x, axis=1)
    df = df[df['Country'] == country_name]
    df = df.where(pd.notnull(df), None)
    df_dict = df.to_dict(orient='records')
    import_file = {"data":df_dict}
    print ("length of downloaded file: ",len(df_dict))
    
    #Save the file as a json with the name you specified
    with open(New_Name+ '.json', 'w') as fp:
        json.dump(import_file, fp)
    
    return import_file

# Set User Defined Parameters

#### Change the below parameters based on the country being processed and the full location of your download folder


In [5]:
# replace country name
country_name = 'Kuwait'
# Specify where the data usually loads and choose the last downloaded file
folder_path = "C:/Users/10197378/Downloads"
# Specify where the chromedriver is. if its in the same folder you only need to specify the executable name as seen below
chrome_driver_path = "C:/Users/10197378/chromedriver.exe"

#### CHANGE THE FOLLOWING ONLY IF YOU RECIEVE AN ERROR

##### The below are xpath locators, if you recieve an error you may need to change one locator. When you are copying a locator from chrome's inspect tab, make sure to resize the inspect tab to the smallest size possible. The Xpath will change if the page resizes for any reason, this is why we reduce the size of the inspector tab.

In [10]:
# add Locators loc_s is for the export button, and loc_c is for the "export to excel" button within the main export button menue
loc_s = '/html/body/div/div/div/div[1]/div[1]/div/div[3]/div/div[2]/div/button'
loc_c = '/html/body/div/div/div/div[1]/div[1]/div/div[3]/div/div[2]/div[2]/div[2]/div[2]/li[2]'

#loc_s = '/html/body/div/div/div/div[1]/header/div/div/div[2]/div/div[1]/div/button/span[1]/div/svg'
#loc_c = '/html/body/div/div/div/div[1]/header/div/div/div[2]/div/div[1]/div[2]/div[2]/div[2]/li[2]/span[1]'

# add locators for sdg page

# cog_wheel and other filter related objects
cog_wheel = "/html/body/div/div/div/div[1]/div/div[3]/div[2]/div[1]/div/div/div/div[2]/div[1]/button[1]"
dimensions_button = "/html/body/div/div/div/div[1]/div/div[3]/div[2]/div[1]/div/div/div/div[1]/div/div/div/div/div/div/ul/div"
country_edit_button = "/html/body/div/div/div/div[1]/div/div[3]/div[2]/div[1]/div/div/div/div[1]/div/div/div/div/div[1]/div/ul/div[2]/div/div/div/div[2]/div[2]/div[4]/div/button[1]"
deselect_all_button = "//span[text()='Deselect all']"
filter_dialogue_window = "/html/body/div/div/div/div[5]/div[2]/div[2]/div/div[2]"  # xpath for list of countries 
select_button = "//span[text()='Select']"
apply_button = "//span[text()='Apply']"

# SDG add Locators loc_s is for the export button, and loc_c is for the "export to excel" button within the main export button menue
loc_s_sdg = "/html/body/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[2]/div/button"
loc_c_sdg = "/html/body/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[2]/div[2]/div[2]/div[2]/li[2]"


# Call Main Function With The Different Data URLs

In [11]:
#Add the URLs
URLs=[
    'https://data.unescwa.org/portal/c78f8929-babc-40d9-8d9b-a5438396d3a7', #National Accounts
    'https://data.unescwa.org/portal/37e07376-dcab-4c77-b25d-b63c6c1c0d4b', #Finance
    'https://data.unescwa.org/portal/a2f766e6-f2c0-4444-aa73-ccd0d7abf05d', #Trade
    'https://data.unescwa.org/portal/79a73daa-8d9e-47dd-b1ee-bfed2b22124e', #Transport
    'https://data.unescwa.org/portal/ae81b7b7-05b5-4b39-b87a-1243bef7b232', #Industry

#     'https://data.unescwa.org/portal/CPI',                                  #CPI
#     'https://data.unescwa.org/portal/a400b071-fd51-4aac-87ae-175627339bdd', #ICP
#     'https://data.unescwa.org/portal/gems_data',                            #GEMS

    'https://data.unescwa.org/portal/7bae8f12-f543-407f-9afa-29471d6b7c6d', #Population
    'https://data.unescwa.org/portal/e68647fb-ea6d-488d-a6f5-2024b080c2cc', #Household Budget
    'https://data.unescwa.org/portal/8c972cac-a80c-4bd4-8208-74c6a092e225', #Health
    'https://data.unescwa.org/portal/e7d41253-2cf5-4f3b-ba5f-6c45b8af1f88', #Education
    'https://data.unescwa.org/portal/69b86687-53d8-4c50-b5d7-b6c96df42d0b', #Labor
    'https://data.unescwa.org/portal/e28b867b-13b6-4d97-ad5e-85264879c2ef', #Environment
    'https://data.unescwa.org/portal/ed5c2876-01e0-492b-9fad-49702ec6d934', #Energy
    'https://data.unescwa.org/share/618997eb-93eb-466c-b768-50557b5ae7b2' #SDG

    ]

#Names of the new saved data

Names=[
    'National Accounts',
    'Finance',
    'Trade',
    'Transport',
    'Industry',
#     'CPI',
#     'ICP',
#     'GEMS',
    'Population',
    'Household Budget',
    'Health',
    'Education',
    'Labor',
    'Environment',
    'Energy',
     'SDG'

    
    ]

   
for i in range(len(URLs)):
    print("Starting with Document: " ,i)
    driver = webdriver.Chrome('C:/Users/10197378/chromedriver.exe')
    driver.maximize_window()
    if Names[i] == 'SDG':
        Upload_Data_as_JSON(driver,URLs[i],country_name,loc_s_sdg,loc_c_sdg,folder_path, Names[i],True)
    else:    
        Upload_Data_as_JSON(driver,URLs[i],country_name,loc_s,loc_c,folder_path, Names[i],False)
    
    driver.close()
    print("Finished document: " , i)


Starting with Document:  0


C:\Users\10197378\AppData\Local\Temp\ipykernel_20652\4216131609.py:50: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/10197378/chromedriver.exe')


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'